# Demonstrace metody LIME (část 1/3)

V této ukázce natrénujeme jednoduchý klasifikátor, který bude klasifikovat příspěvky z diskuzních skupin o ateismu a o křesťanství na základě obsažených slov.
K tomu využijeme knihovnu pro strojové učení [scikit-learn](https://scikit-learn.org).
Následně použijeme metodu [LIME](https://github.com/marcotcr/lime) k vysvětlení predikcí natrénovaného klasifikátoru.

In [ ]:
import lime
import sklearn
import sklearn.ensemble
import sklearn.metrics

## Načtení dat a natrénování klasifikátoru

Použijeme data z [20 newsgroups datasetu](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html), konkrétně diskuzní skupiny o ateismu a o křesťanství.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christianity']

Prohlédněme si data:

In [ ]:
from IPython.display import display, HTML
import html

# Pomocná funkce pro vypsání.
def display_post(id):
    display(HTML(f"""
        <b>id: {html.escape(str(id))}</b><br>
        <b>class: {html.escape(class_names[newsgroups_train.target[id]])}</b>
        <pre>{html.escape(newsgroups_train.data[id])}</pre>
        """))

In [ ]:
display_post(id=50)

In [ ]:
display_post(id=100)

Pro natrénování klasifikátoru je potřeba reprezentovat slova jako vektory. Zde použijeme metodu [term frequency–inverse document frequency](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction). Zjednodušeně řečeno, každé slovo reprezentujeme vektorem o délce počtu trénovacích příspěvků z diskuzních skupin, který pro každý trénovací příspěvek vyjadřuje, jak je v něm dotyčné slovo významné na základě četnosti.

In [ ]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=True)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

Natrénujeme [náhodné rozhodovací stromy](https://scikit-learn.org/stable/modules/ensemble.html).

In [ ]:
classifier = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
classifier.fit(train_vectors, newsgroups_train.target)

## Vyhodnocení klasifikátoru pomocí $F_1$ skóre

$F_1$ skóre měří výkon klasifikátoru na testovacích datech a nabývá hodnot 0 až 1 (nejlepší). Jak vidíme níže, $F_1$ skóre natrénovaného klasifikátoru je velmi vysoké.

In [ ]:
pred = classifier.predict(test_vectors)
f1_score = sklearn.metrics.f1_score(newsgroups_test.target, pred, average='binary')

print(f"F1 skóre: {f1_score}")

## Vysvětlení predikcí pomocí metody LIME

Vysvětlení vytváří instance `LimeTextExplainer`.

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

`LimeTextExplainer` funguje s jakýmkoliv klasifikátorem, který implementuje metodu s výstupem obdobným `predict_proba`. Jako vstup klasifikátoru ale přepokládá text a nikoliv vektory. Za tímto účelem lze využít pomocnou `pipeline`.

In [ ]:
from sklearn.pipeline import make_pipeline
raw_text_classifier = make_pipeline(vectorizer, classifier)

# Jen pro demonstraci výstupu.
print(raw_text_classifier.predict_proba([newsgroups_test.data[0]]))

Zvolíme libovolný příspěvek. Podíváme se, jak je klasifikován naším modelem.

In [ ]:
idx = 83 # Zvolený příspěvek.
probs = raw_text_classifier.predict_proba([newsgroups_test.data[idx]])
print(f"Pravděpodobnost: ({class_names[0]}) = {probs[0, 0]}")
print(f"Pravděpodobnost: ({class_names[1]}) = {probs[0, 1]}")
print(f"Skutečná třída: {class_names[newsgroups_test.target[idx]]}")

Příspěvek je klasifikován správně.

Nyní zkusíme vygenerovat vysvětlení predikce tohoto příspěvku. Maximální počet příznaků ve vysvětlení omezíme na 10.

Pozn.: Níže použitá metoda `show_in_notebook` je jen jednou z možností, jak získat či vizualizovat vysvětlení.

In [ ]:
explanation = explainer.explain_instance(newsgroups_test.data[idx], raw_text_classifier.predict_proba, num_features=10)
explanation.show_in_notebook(text=True)

Metoda LIME se naučí lineární model aproximující náš klasifikátor v okolí predikovaného příspěvku. Graf ve druhém sloupci zobrazuje váhy tohoto modelu pro jednotlivá slova. Za předpokladu linearity klasifikátoru by odstranění všech výskytů slov "NNTP" a "Host" mělo snížit pravděpodobnost třídy "atheism" o součet jejich vah. Podívejme se na výstup klasifikátoru pro takto upravený příspěvek.

In [ ]:
tmp = test_vectors[idx].copy()
tmp[0, vectorizer.vocabulary_['nntp']] = 0
tmp[0, vectorizer.vocabulary_['host']] = 0

probs = raw_text_classifier.predict_proba([newsgroups_test.data[idx]])
probs2 = classifier.predict_proba(tmp)
d = {str(k): v for k, v in explanation.as_list()}
print(f"Původní pravděpodobnost: ({class_names[0]}) = {probs[0, 0]:0.2f}")
print(f"Nová pravděpodobnost: ({class_names[0]}) = {probs2[0, 0]:0.2f}")
print(f"Předpokládané snížení: {d['NNTP'] + d['Host']:0.2f}")
print(f"Skutečně snížení: {probs2[0, 0] - probs[0, 0]:0.2f}")
print(f"Skutečná třída: {class_names[newsgroups_test.target[idx]]}")

Skutečný model není lineární a snížení pravděpodobnosti tak pravděpodobně neodpovídá zcela předpokladu (záleží, jak se model natrénoval), ale je podobně významné.

### Vizualizace predikcí

**Otázka:** Prohlédněte si vysvětlení predikcí (delší texty vyžadují skrolování v okénku textu). Rozpoznávač měl vysoké $F_1$ skóre, tedy na testovacích datech si vede velmi dobře. Zaměřuje se ale skutečně na slova týkající se klasifikovaných témat? Co lze zlepšit?

In [ ]:
idx = 83
explanation = explainer.explain_instance(newsgroups_test.data[idx], raw_text_classifier.predict_proba, num_features=10)
print(f"Skutečná třída: {class_names[newsgroups_test.target[idx]]}")
explanation.show_in_notebook(text=True)

In [ ]:
idx = 50
explanation = explainer.explain_instance(newsgroups_test.data[idx], raw_text_classifier.predict_proba, num_features=10)
print(f"Skutečná třída: {class_names[newsgroups_test.target[idx]]}")
explanation.show_in_notebook(text=True)